# Define functions for crawler video

## Crawl video and audio

In [90]:
import os
import yt_dlp
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip, ffmpeg_extract_audio

output_dir = "data"
os.makedirs(output_dir, exist_ok=True)

def crawler_video(video_id, year):
    try:
        video_url = "https://youtu.be/" + video_id
        # Tải xuống 1 phút đầu video + audio
        temp_video_path = os.path.join(output_dir+"/video/" + str(year) + "/tmp_" + str(video_id) + ".mp4")
        video_path = os.path.join(output_dir+"/video/" + str(year) + "/" + str(video_id) + ".mp4")
        download_opts = {
            "format": "mp4",
            "outtmpl": temp_video_path,
            "external_downloader": "ffmpeg",
            "external_downloader_args": [f"-ss 0 -i {temp_video_path} -t 60 -c copy {video_path}"]
        }
        with yt_dlp.YoutubeDL(download_opts) as ydl:
            ydl.download([video_url])
        
        ffmpeg_extract_subclip(temp_video_path, 0, 60, outputfile=video_path)
        # os.system(f"ffmpeg -ss 0 -i {temp_video_path} -t 60 -c copy {video_path}")
        os.remove(temp_video_path)

        # Tách audio ra khỏi video
        audio_path = os.path.join(output_dir+"/audio/" + str(year) + "/" + str(video_id) + ".mp3")
        
        ffmpeg_extract_audio(video_path, audio_path)
        
        return True
    except Exception as e:
        print("❌ Không thể tải video")
        print(e)
        return False

## Crawl thumbnail

In [91]:
def crawler_thumbnail(video_id, year):
    video_url = "https://youtu.be/" + video_id
    try:
        # Tạo thư mục thumbnail nếu chưa có
        save_dir = os.path.join(output_dir, "thumbnail", year)
        os.makedirs(save_dir, exist_ok=True)

        # Cấu hình yt-dlp để thu thập thông tin video
        ydl_opts = {
            "quiet": True,
            "extract_flat": False,
            "nocheckcertificate": True,
            "format": "bestaudio/best",
            "postprocessors": [
                {"key": "FFmpegExtractAudio", "preferredcodec": "mp3", "preferredquality": "192"},
            ],
            "outtmpl": {"default": os.path.join(output_dir, "%(id)s.%(ext)s")},
            "writethumbnail": True,  # Chỉ viết metadata thumbnail, không tải
            "skip_download": True,   # Không tải audio/video
        }

        # Thu thập thông tin video
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(video_url, download=False)

        # ID video thực tế
        video_id = info["id"]

        # Lọc danh sách thumbnail
        thumbnail_urls = sorted(
            [thumb for thumb in info.get("thumbnails", []) if "height" in thumb],
            key=lambda x: x["height"], reverse=True
        )[:3]

        # Nếu không có height, lấy 3 ảnh đầu tiên
        if not thumbnail_urls:
            thumbnail_urls = info.get("thumbnails", [])[:3]

        # Tải xuống các thumbnail
        for i, thumb in enumerate(thumbnail_urls):
            thumb_url = thumb["url"]
            response = requests.get(thumb_url, stream=True)
            if response.status_code == 200:
                file_path = os.path.join(save_dir, f"{video_id}_thumb{i+1}.jpg")
                with open(file_path, "wb") as file:
                    file.write(response.content)
                print(f"✅ Đã tải {file_path}")
            else:
                print(f"❌ Không thể tải {thumb_url}")

    except Exception as e:
        print(f"❌ Lỗi: {e}")

## Crawl title & tag

In [ ]:
def get_video_info(video_id):
    video_url = f"https://youtu.be/{video_id}"
    ydl_opts = {
        'quiet': True,
        'no_warnings': True,
        'extract_flat': True,
        'skip_download': True,
    }
    
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(video_url, download=False)
        
    title = info.get('title', '')
    tags = info.get('tags', [])[:5]  # Get top 5 tags
    
    return title, tags


# Crawling

In [92]:
category = "test"

In [93]:
import pandas as pd
from enum import Enum

class CrawlStatus(Enum):
    NOT_YET="NOT_YET"
    DONE="DONE"

# Load a parquet file
path = f'data/{category}.parquet'
df = pd.read_parquet(path)

if "crawl_status" not in df.columns:
    df["crawl_status"] = CrawlStatus.NOT_YET.value
    print("Created crawl_status")

In [96]:
from tqdm import tqdm

for i in tqdm(range(len(df))):
    if df["crawl_status"][i] == CrawlStatus.DONE.value:
        continue
    is_crawl_success = crawler_video(df["id"][i], category)
    if is_crawl_success:
        df.loc[i, "crawl_status"] = CrawlStatus.DONE.value
    if (i % 100 == 10):
        print("Processing {}".format(i))
        df.to_parquet(path)

[youtube] Extracting URL: https://youtu.be/FIkKcpltyfc
[youtube] FIkKcpltyfc: Downloading webpage
[youtube] FIkKcpltyfc: Downloading tv client config
[youtube] FIkKcpltyfc: Downloading player 7d1d50a6
[youtube] FIkKcpltyfc: Downloading tv player API JSON
[youtube] FIkKcpltyfc: Downloading ios player API JSON
[youtube] FIkKcpltyfc: Downloading m3u8 information
[info] FIkKcpltyfc: Downloading 1 format(s): 18
[download] Destination: data/video/test/tmp_FIkKcpltyfc.mp4
[download] 100% of  163.71MiB in 00:00:05 at 31.50MiB/s    
MoviePy - Running:
>>> /Users/vuongnguyen/Downloads/Github/Youtube-Engagement-Deep-Multimodal-Fusion/.venv/lib/python3.10/site-packages/imageio_ffmpeg/binaries/ffmpeg-macos-aarch64-v7.1 -y -ss 0.00 -i data/video/test/tmp_FIkKcpltyfc.mp4 -t 60.00 -map 0 -vcodec copy -acodec copy -copyts data/video/test/FIkKcpltyfc.mp4
MoviePy - Command successful
MoviePy - Running:
>>> /Users/vuongnguyen/Downloads/Github/Youtube-Engagement-Deep-Multimodal-Fusion/.venv/lib/python3.10/

[download] Got error: 385463 bytes read, 10097016 more expected


❌ Không thể tải video
[download] Got error: 385463 bytes read, 10097016 more expected
[youtube] Extracting URL: https://youtu.be/ZqqYjlknlSA
[youtube] ZqqYjlknlSA: Downloading webpage
[youtube] ZqqYjlknlSA: Downloading tv client config
[youtube] ZqqYjlknlSA: Downloading player 7d1d50a6
[youtube] ZqqYjlknlSA: Downloading tv player API JSON
[youtube] ZqqYjlknlSA: Downloading ios player API JSON
[youtube] ZqqYjlknlSA: Downloading m3u8 information
[info] ZqqYjlknlSA: Downloading 1 format(s): 18
[download] Destination: data/video/test/tmp_ZqqYjlknlSA.mp4
[download] 100% of   21.66MiB in 00:00:01 at 16.88MiB/s    
MoviePy - Running:
>>> /Users/vuongnguyen/Downloads/Github/Youtube-Engagement-Deep-Multimodal-Fusion/.venv/lib/python3.10/site-packages/imageio_ffmpeg/binaries/ffmpeg-macos-aarch64-v7.1 -y -ss 0.00 -i data/video/test/tmp_ZqqYjlknlSA.mp4 -t 60.00 -map 0 -vcodec copy -acodec copy -copyts data/video/test/ZqqYjlknlSA.mp4
MoviePy - Command successful
MoviePy - Running:
>>> /Users/vuongn

ERROR: [youtube] nVuFgcS_HwU: Private video. Sign in if you've been granted access to this video. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies


❌ Không thể tải video
ERROR: [youtube] nVuFgcS_HwU: Private video. Sign in if you've been granted access to this video. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies
[youtube] Extracting URL: https://youtu.be/iJkfPcDIeMM
[youtube] iJkfPcDIeMM: Downloading webpage
[youtube] iJkfPcDIeMM: Downloading tv client config
[youtube] iJkfPcDIeMM: Downloading player 7d1d50a6
[youtube] iJkfPcDIeMM: Downloading tv player API JSON
[youtube] iJkfPcDIeMM: Downloading ios player API JSON


...

[download] Got error: HTTPSConnectionPool(host='rr6---sn-8qj-nbo67.googlevideo.com', port=443): Read timed out.


❌ Không thể tải video
[download] Got error: HTTPSConnectionPool(host='rr6---sn-8qj-nbo67.googlevideo.com', port=443): Read timed out.
[youtube] Extracting URL: https://youtu.be/zhKdt0pOvno
[youtube] zhKdt0pOvno: Downloading webpage
[youtube] zhKdt0pOvno: Downloading tv client config
[youtube] zhKdt0pOvno: Downloading player 7d1d50a6
[youtube] zhKdt0pOvno: Downloading tv player API JSON
[youtube] zhKdt0pOvno: Downloading ios player API JSON
[youtube] zhKdt0pOvno: Downloading m3u8 information
[info] zhKdt0pOvno: Downloading 1 format(s): 18
[download] Destination: data/video/test/tmp_zhKdt0pOvno.mp4
[download]  19.9% of  147.59MiB at    5.38MiB/s ETA 00:21  

[download] Got error: HTTPSConnectionPool(host='rr4---sn-8qj-nbo67.googlevideo.com', port=443): Read timed out.


❌ Không thể tải video
[download] Got error: HTTPSConnectionPool(host='rr4---sn-8qj-nbo67.googlevideo.com', port=443): Read timed out.
[youtube] Extracting URL: https://youtu.be/o0sDwCjQ344
[youtube] o0sDwCjQ344: Downloading webpage
[youtube] o0sDwCjQ344: Downloading tv client config
[youtube] o0sDwCjQ344: Downloading player 7d1d50a6
[youtube] o0sDwCjQ344: Downloading tv player API JSON
[youtube] o0sDwCjQ344: Downloading ios player API JSON
[youtube] o0sDwCjQ344: Downloading m3u8 information
[info] o0sDwCjQ344: Downloading 1 format(s): 18
[download] Destination: data/video/test/tmp_o0sDwCjQ344.mp4
[download] 100% of   95.06MiB in 00:00:39 at 2.40MiB/s     
MoviePy - Running:
>>> /Users/vuongnguyen/Downloads/Github/Youtube-Engagement-Deep-Multimodal-Fusion/.venv/lib/python3.10/site-packages/imageio_ffmpeg/binaries/ffmpeg-macos-aarch64-v7.1 -y -ss 0.00 -i data/video/test/tmp_o0sDwCjQ344.mp4 -t 60.00 -map 0 -vcodec copy -acodec copy -copyts data/video/test/o0sDwCjQ344.mp4
MoviePy - Command

KeyboardInterrupt: 

In [76]:
df = pd.read_parquet(path)